In [ ]:
!pip install yfinance tqdm pytz --quiet

In [ ]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import pytz

X 데이터 수집

In [ ]:
# ============================================
# 수집할 종목 리스트 정의
# ============================================
symbols = ['^VIX','GC=F','SHY']

In [ ]:
# =============================
# 2. 시계열 데이터 수집 (뉴욕시간 기준)
# =============================
START = "2025-11-10"
END   = "2025-11-19"

In [ ]:
print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

In [ ]:
# UTC로 지정 → 뉴욕시간으로 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# '세션 날짜'를 계산: 전날 저녁 라벨 → 다음날 거래일로 이동
session_dates = (data.index + pd.Timedelta(days=1)).normalize()

# '09:30' (개장시각) 타임스탬프로 교체
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)

# 이걸 인덱스로 지정
data.index = session_times

print("인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료")
print(data.index.min(), "→", data.index.max())

In [ ]:
print("데이터 레벨명:", data.columns.names)   # ['Attributes', 'Symbols'] 형태
print("기간:", data.index.min().date(), "~", data.index.max().date())

In [ ]:
# 필요한 컬럼 정리
ohlcv = data[["High", "Low","Volume"]].copy()
ohlcv.columns = [f"{ticker}_{col}" for col, ticker in ohlcv.columns]

In [ ]:
# 변동성(High-Low)
volatility = pd.DataFrame(index=ohlcv.index)
for sym in symbols:
    h, l = f"{sym}_High", f"{sym}_Low"
    if h in ohlcv.columns and l in ohlcv.columns:
        volatility[f"{sym}_Volatility"] = ohlcv[h] - ohlcv[l]

price_features = pd.concat([ohlcv, volatility], axis=1).sort_index()

In [ ]:
price_features.info()

In [ ]:
price_features.columns

In [ ]:
price_features = price_features.drop(['GC=F_High', 'SHY_High', '^VIX_High', 'GC=F_Low', 'SHY_Low', '^VIX_Low','SHY_Volume', '^VIX_Volume'], axis=1)

In [ ]:
price_features.info()

In [ ]:
price_features

In [ ]:
import pandas as pd

# 1) CSV 불러오기
path = "../output_data/투자지표_555_평균.csv"
df_ind = pd.read_csv(path)

# 2) PER(배) 마지막 값 가져오기
last_per = df_ind["PER(배)"].dropna().iloc[-1]

# 3) price_features에 새로운 컬럼으로 추가
price_features["PER(배)"] = last_per

print(price_features.tail())

In [ ]:
price_features

Y데이터 수집

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# ============================================
# 1) 종목 리스트
# ============================================
symbols = ["SPY"]

# ============================================
# 2) 시계열 데이터 수집
# ============================================
START = "2025-11-10"
END   = "2025-11-19"

print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

# UTC → 뉴욕시간 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# 전날 가격을 다음날 개장시간(09:30)에 반영
session_dates = (data.index + pd.Timedelta(days=1)).normalize()
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)
data.index = session_times

print("NY 개장 시간으로 라벨 조정 완료")
print("기간:", data.index.min().date(), "~", data.index.max().date())

# ============================================
# 3) MultiIndex → 단일 컬럼으로 변환
# ============================================
data.columns = [f"{col}_{sym}" for col, sym in data.columns]

# ============================================
# 4) 필요한 컬럼만 선택
# ============================================
ohlcv = data[["Close_SPY"]].copy()

ohlcv[["y_target"]] = ohlcv[["Close_SPY"]]

# 첫 번째 행은 값이 없으니 보통은 드랍
# ohlcv = ohlcv.dropna().copy()

# ============================================
# 5) Date 컬럼 생성
# ============================================
ohlcv["Date"] = ohlcv.index.tz_localize(None).normalize().date
ohlcv.reset_index(drop=True, inplace=True)

# Date 컬럼을 맨 앞으로 이동
ohlcv = ohlcv[["Date"] + [col for col in ohlcv.columns if col != "Date"]]

# ============================================
# 6) 출력
# ============================================
print(ohlcv)

In [ ]:
ohlcv = ohlcv.drop(['Close_SPY'], axis=1)

In [ ]:
ohlcv

X,Y 데이터 병합

In [ ]:
# 1) pf(Date 컬럼을 인덱스로 설정)
pf = price_features.copy()
pf.index = pd.to_datetime(pf.index).tz_localize(None)   # 타임존 제거
pf = pf.rename_axis("Date").reset_index()               # index → Date 컬럼
pf["Date"] = pd.to_datetime(pf["Date"]).dt.date         # 순수 날짜
pf = pf.set_index("Date")                               # Date를 인덱스로 설정

# 2) ohlcv2(Date를 인덱스로 가진 형태로 통일)
ohlcv2 = ohlcv.copy()
ohlcv2.index = pd.to_datetime(ohlcv2.index).tz_localize(None)  # tz 제거
ohlcv2.index = ohlcv2.index.date                                # date 형태로 변환

# 3) 인덱스(Date) 기준 병합 (join)
merged_df = pf.join(ohlcv2, how="inner")   # 필요하면 inner로 변경 가능

print(merged_df)

In [ ]:
merged_df

In [ ]:
merged_df_clean = merged_df.dropna()

In [ ]:
merged_df_clean

In [ ]:
# 1. 'y_target'에 로그 변환을 적용하여 'y_target_log' 생성
merged_df_clean['y_target_log'] = np.log(merged_df_clean['y_target'])
# 2. 타겟(y) 정의
y = merged_df_clean['y_target_log']

# 3. X 정의 (모든 피처 포함)
X = merged_df_clean.drop(columns=['y_target', 'y_target_log','PER(배)'])

In [ ]:
# -------------------------------
# 1) 자동 로그 변환 함수 (그대로 사용)
# -------------------------------
def auto_log_transform_by_value(merged_df_clean):
    print(f"--- 자동 로그 변환 시작 (값 범위 기반) ---")

    df_transformed = merged_df_clean.copy()
    transformed_cols = []

    numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        col_data = df_transformed[col]
        col_min = col_data.min()

        if col_min < 0:
            print(f"[{col}] 건너뜀: 음수 값이 포함되어 있습니다. (최소값: {col_min:.2f})")
            continue

        elif col_min < 1:
            df_transformed[col] = np.log1p(col_data)
            transformed_cols.append(col)
            print(f"[{col}] 변환 적용 (log1p): 최소값={col_min:.2f} (< 1)")

        else:
            df_transformed[col] = np.log(col_data)
            transformed_cols.append(col)
            print(f"[{col}] 변환 적용 (log): 최소값={col_min:.2f} (>= 1)")

    print(f"--- 총 {len(transformed_cols)}개 열 변환 완료 ---")

    return df_transformed, transformed_cols

In [ ]:
X_transformed, transformed_list_X = auto_log_transform_by_value(X)

print("\n[X 변환된 컬럼 왜도]")
print(X_transformed[transformed_list_X].skew())

In [ ]:
merged_df_clean

In [ ]:
X_transformed['PER(배)'] = merged_df_clean['PER(배)']

In [ ]:
X_transformed['y_target_log'] = merged_df_clean['y_target_log']

In [ ]:
X_transformed

In [ ]:
X_transformed.info()

In [ ]:
X_transformed.to_csv("../../../예측/예측샘플/log샘플데이터.csv")